<div class="alert alert-success" role="alert">


### Until July 18: Course Evaluation

We invite you to take part in the evaluation of this course. 
Visit the [**link**](https://evaluation.tu-darmstadt.de/evasys/online.php?pswd=6FTSJ) to access the evaluation form.

**If you enjoy the course**:<br> Let us know what’s working well. Your positive feedback helps us understand which aspects are effective and should be retained in future courses.

**If you have suggestions or criticism**:<br> This is your chance to highlight areas for improvement. Constructive feedback helps us tailor the course to better meet your needs.  

- *Speak to Us Directly*:<br>
If you are unsatisfied with any aspect of the course, we encourage you to speak to us directly. We can address your concerns faster than the central processing of the evaluation results. Your input matters, and we’re here to help you.  

# Exercise: (Py)Calphad

After working with atomistic simulations, we now shift our focus to a thermodynamic approach. While atomistic methods provide detailed insights into atomic interactions and short-range phenomena, they often fall short in efficiently predicting long-range phase stability and equilibria in complex, multicomponent systems. To address these challenges, we now move to the **CALPHAD** method.

CALPHAD, which stands for *CALculation of PHAse Diagrams*, is a computational methodology used to model and predict phase equilibria and thermodynamic properties of multicomponent materials systems. *It combines experimental data, thermodynamic principles, and mathematical models to construct databases that describe how materials behave under varying conditions of temperature, pressure, and composition*.

Originally developed to aid in alloy design and metallurgy, CALPHAD has since expanded into ceramics, polymers, and other complex materials. By providing accurate phase diagrams and property predictions, CALPHAD supports materials research, engineering, and development by reducing the need for extensive experimental trials. Its models are widely used in both academic research and industrial applications such as process optimization and new material design.

Before addressing the main tasks, let’s go through a quick tutorial to get familiar with the basic features of the CALPHAD approach.

<div style="background-color:#e6f0ff; padding: 10px; border-left: 5px solid #3399ff; font-family: sans-serif;">
  <p>
    We gratefully acknowledge the developers of 
    <a href="https://github.com/pycalphad/pycalphad" target="_blank" style="color:#0056b3; text-decoration:none;">
      PyCalphad
    </a>. 
  </p>
</div>

In [ ]:
! pip install pycalphad

from pycalphad import Database, calculate, equilibrium, variables as v, Model, binplot

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_context("talk")

import numpy as np

In [ ]:
# EXECUTE THIS CELL
# Some utility functions

def print_constituents(phase_name, db):
    constituents = db.phases[phase_name].constituents
    print(f"Constituents for phase '{phase_name}':")
    for i, sublattice in enumerate(constituents):
        species_names = [sp.name for sp in sublattice]
        print(f"\tSublattice {i+1}: {', '.join(species_names)}")


def summarize_equilibrium_result(eq):
    """
    Print a complete summary of a PyCalphad equilibrium() result.
    Handles single-condition grids and prints:
      - Conditions
      - Total Gibbs energy
      - Phases
      - Phase amounts
      - Compositions (X)
      - Chemical potentials (MU)
      - Site fractions (Y)
    """
    print("Equilibrium Summary")
    print("-" * 60)

    # Identify scalar condition axes (e.g., T, P, X_NI)
    cond_axes = [ax for ax in eq.sizes if ax not in ['vertex', 'component', 'internal_dof']]
    coords = {name: eq.coords[name].values.item() for name in cond_axes}
    for k, v_ in coords.items():
        print(f"{k}:\t\t\t{v_}")

    # Total molar Gibbs energy
    GM = eq.GM.values.item()
    print(f"\nTotal molar Gibbs energy:\t{GM:.3f} J/mol\n")

    # Phase-level info
    phases = eq.Phase.values
    phase_amounts = eq.NP.values
    X = eq.X.values
    Y = eq.Y.values
    MU = eq.MU.values
    components = list(eq.component.values)
    n_vertices = eq.sizes['vertex']
    n_internal_dof = eq.sizes['internal_dof']

    # Print chemical potentials
    print("Chemical potentials (μ) [J/mol]:")
    for i, comp in enumerate(components):
        print(f"  μ({comp}):\t\t{MU[0, 0, 0, 0, i]:.3f}")
    print()

    # Print info per vertex (stable phases)
    for i in range(n_vertices):
        phase = phases[0, 0, 0, 0, i]
        if isinstance(phase, str) and phase != '':
            print(f"Phase: {phase}")
            print(f"  Amount: {phase_amounts[0, 0, 0, 0, i]:.4f} mol")
            print("  Composition (X):")
            for j, comp in enumerate(components):
                x_val = X[0, 0, 0, 0, i, j]
                print(f"    X({comp}): {x_val:.4f}")
            print()

    print("-" * 60)

## Opening and Exploring a Database

Before performing any calculation in PyCalphad, you need to define four key things:

1. A thermodynamic database

2. The chemical components (including `'VA'` for vacancies)

3. The phases to include in the calculation

4. The calculation conditions (e.g., temperature, composition)

To load a database, use the `Database()` command:

In [ ]:
db = Database('Cr-Fe-Ni.tdb')

> **Note:** The `.tdb` file must be located in the same directory as your Jupyter notebook for this to work.

## Exploring Database Contents

Thermodynamic databases contain a lot of useful information to help you set up your calculation. Three particularly helpful things to examine are:

- The elements defined in the database

- The available phases

- The constituents of a specific phase

You can inspect them using the following commands:

In [ ]:
print("The elements in this database are:")
print(db.elements)
print()
print("The phases in this database are:")
print(list(db.phases.keys()))
print()

In [ ]:
for phase in list(db.phases.keys()):
    print_constituents(phase, db)
    print()

## Setting Conditions and Running Equilibrium Calculations

In thermodynamic calculations, you must specify $c + 2$ conditions for $c$ components. However, in PyCalphad, calculations are performed per mole of system (i.e., $n = 1$ is assumed by default), so you only need to define $c + 1$ conditions.

Each condition—pressure, temperature, and composition—is defined using variables from `pycalphad.variables`, usually accessed with the `v.` prefix:

- **Pressure**: `v.P`, in Pascals (Pa). Most databases assume a standard pressure of $101325$ Pa ($1$ atm), and often do not include explicit pressure dependence.

- **Temperature**: `v.T`, in Kelvin (K). Databases are generally valid between $\approx 298.15$ K and $6000$ K.

- **Composition**: `v.X('NI')`, which sets the mole fraction of a component (e.g., `'NI'` for nickel).

These variables are typically used with the `equilibrium()` function, which computes stable phase equilibria under given conditions.

The table below shows the variable names used to access results returned by `equilibrium()`, which are also property keywords you can use later on with the function `calculate()`:

| Property                      | Keyword                    |
|------------------------------|----------------------------|
| Molar Enthalpy               | `'HM'`                     |
| Mixing Enthalpy              | `'HM_MIX'`                 |
| Molar Gibbs Energy           | `'GM'`                     |
| Mixing Gibbs Energy          | `'GM_MIX'`                 |
| Molar Entropy                | `'SM'`                     |
| Heat Capacity (total)        | `'heat_capacity'`          |
| Degree of Ordering           | `'degree_of_ordering'` or `'DOO'` |
| Molar Heat Capacity          | `'CPM'`                    |
| Mixing Heat Capacity         | `'CPM_MIX'`                |
| Excess Mixing Energy         | `'excess_mixing_energy'`   |

Use these keywords to specify which thermodynamic quantity you would like to calculate.

When you load a thermodynamic database (like `Cr-Fe-Ni.tdb`), it contains models and data for many elements and phases, but you're free to focus on a subset (e.g., Cr) for your calculations or plotting.
You can restrict the component list manually:
```python
components = ['CR', 'VA']  # Always include 'VA' for vacancies
```
This tells PyCalphad that you want to perform calculations for the Cr unary system (i.e., pure chromium). The 'VA' component (vacancy) must always be included when working with substitutional phases in CALPHAD models, even if you're working with just one real element.

# Task (a): The Fe-Ni system

In this exercise, you will use PyCalphad to calculate the equilibrium phase assemblage of the Fe–Ni binary system under specified conditions.


1. Load the Thermodynamic Database
   Replace `'...'` with the actual filename that contains the thermodynamic data for the Fe–Ni system.

2. Define the Components

   * Use uppercase for chemical symbols (e.g., `'FE'`, `'NI'`)
   * Always include `'VA'` (the vacancy species), which is required for substitutional solution models.

3. Set the Calculation Conditions

   * Pressure: `101325` Pa (equivalent to 1 atm)
   * Temperature: `500` K
   * Composition: Mole fraction of Ni = `0.5` (i.e., a 50–50 Fe–Ni alloy)

4. Run the Calculation
   After executing the code, the `summarize_equilibrium_result(eq)` function (already defined in the notebook) will output a summary of the equilibrium:

   * Which phases are stable under the given conditions?
   * How much of each phase is present?
   * What is the composition of each phase (in terms of Fe and Ni)?

In [ ]:
# COMPLETE AND EXECUTE THIS CELL
db = Database('...')

# Define the components:
# Capitalization matters! Use all caps (convention in CALPHAD).
# Always include 'VA' (vacancy) as a component.
# Always include your textual entries in single quotes.
components = [...]

# Define the phases to consider:
phases = list(db.phases.keys())

# Define calculation conditions as a dictionary:
# Keys are variables from pycalphad.variables (accessed via 'v.')
conditions = {
    v.P: ...    ,             # Pressure in Pascals 
    v.T: ...    ,               # Temperature in Kelvin
    v.X('NI'): ...           # Mole fraction of Ni (Fe is implied)
}

# Run the equilibrium calculation
eq = equilibrium(db, components, phases, conditions)

summarize_equilibrium_result(eq)

# Task (b): The Fe-Ni phase diagram

In this exercise, you will generate a binary phase diagram for the Fe–Ni system at constant pressure (1 atm) using PyCalphad’s built-in `binplot()` function.

The `binplot()` function performs equilibrium calculations over a grid of temperature and composition values, identifies phase boundaries, and produces a clean binary phase diagram with labeled phase regions.

You have already defined the database, components, and conditions in the previous equilibrium exercise. Now, you will use this setup to visualize how phase stability changes with temperature and composition.

Unlike a single-point equilibrium calculation, `binplot()` requires the `conditions` dictionary to specify ranges using the format:

```python
(start, stop, step)
```

Update the code to reflect the following:

* Pressure: constant at `101325` Pa (1 atm)

* Temperature: from `400` K to `2000` K, in steps of `50` K

* Ni mole fraction: from `0` to `0.6`, in steps of `0.01` (Fe is the remainder in this binary system)

Once complete, running the cell will display the binary phase diagram, showing which phases are stable under different combinations of temperature and composition.

In [ ]:
phases = list(db.phases.keys())

fig = binplot(
    db,
    components,
    phases,
    conditions = {
    v.P: ...    ,              # Pressure in Pascals 
    v.T: ...    ,            # Temperature in Kelvin
    v.X('NI'): ...        # Mole fraction of Ni
    }
).get_figure()

plt.grid()
# Set the size of the figure
fig.set_size_inches(5, 6)

# Task (c): Verify Phase Stability with `equilibrium()`

Use individual `equilibrium()` calculations to verify the stable phases at specific points on the Fe–Ni phase diagram.

For each point, run a separate `equilibrium()` call and examine:

* Which phases are stable

* The phase fractions (amounts)

* The composition of each phase (in terms of Fe and Ni)

Then, compare the output to the corresponding region of the phase diagram generated earlier. 


Points to Investigate:
| X(Ni) | T (K) |
|-------|-------|
| 0.00  |  800  |
| 0.00  | 1700  |
| 0.10  |  600  |
| 0.10  |  800  |
| 0.47  |  700  |
| 0.47  | 1200  |
| 0.47  | 1800  |

In [ ]:
# Sanity check: verify phases with individual equilibrium calculations

db = Database('Cr-Fe-Ni.tdb')

# Define the components:
# Capitalization matters! Use all caps (convention in CALPHAD).
# Always include 'VA' (vacancy) as a component.
components = ['NI', 'FE', 'VA']

# Define the phases to consider:
phases = list(db.phases.keys())

# Define calculation conditions as a dictionary:
# Keys are variables from pycalphad.variables (accessed via 'v.')
conditions = {
    v.P: ...    ,        # Pressure in Pascals (1 atm)
    v.T: ...    ,           # Temperature in Kelvin
    v.X('NI'): ...      # Mole fraction of Ni (Fe is implied as 0.5)
}

# Run the equilibrium calculation
eq = equilibrium(db, components, phases, conditions)

summarize_equilibrium_result(eq)

## Task (d): Calculating Thermodynamic Properties

In this task, you will explore how key thermodynamic properties evolve with temperature for the Fe–Ni binary system at a fixed composition.

Using `equilibrium()` over a temperature range, you will calculate and visualize:

* Molar enthalpy
* Total (equilibrium) heat capacity
* Phase fractions

Specify:

* Pressure: constant at `101325` Pa (1 atm)

* Temperature: from `400` K to `1800` K, in steps of `5` K

* Ni mole fraction: from `0.47` (Fe is the remainder in this binary system)

This analysis helps reveal how phase transitions affect thermodynamic behavior.

> To specify which properties to calculate, refer to the property name table provided in the tutorial. Be sure to use the correct keyword for each quantity. 

- Do you observe any phase transitions in the plots?

- If so, what order are the transitions?

- Based on the phase fractions and property trends, can you identify which phase transitions are occurring?

| Property                      | Keyword                    |
|------------------------------|----------------------------|
| Molar Enthalpy               | `'HM'`                     |
| Mixing Enthalpy              | `'HM_MIX'`                 |
| Molar Gibbs Energy           | `'GM'`                     |
| Mixing Gibbs Energy          | `'GM_MIX'`                 |
| Molar Entropy                | `'SM'`                     |
| Heat Capacity (total)        | `'heat_capacity'`          |
| Degree of Ordering           | `'degree_of_ordering'` or `'DOO'` |
| Molar Heat Capacity          | `'CPM'`                    |
| Mixing Heat Capacity         | `'CPM_MIX'`                |
| Excess Mixing Energy         | `'excess_mixing_energy'`   |

In [ ]:
# === Define Conditions ===

conditions = {
    v.P: ...    ,         # Pressure (1 atm)
    v.T: (...)  , # Temperature sweep from 400 K to 1800 K, step = 5 K
    v.X('NI'): ...      # Fixed Ni composition
}

# === Run Equilibrium Calculations ===

# Heat capacity and phase data
eq_cp = equilibrium(db, components, phases, conditions, output=['heat_capacity'])

# Enthalpy data
eq_hm = equilibrium(db, components, phases, conditions, output=['HM'])

# === Extract Temperature ===
T = eq_cp['T'].values

# === Extract Enthalpy ===
Hm = np.squeeze(eq_hm['HM'].values)

# === Extract Heat Capacity ===
Cp = np.squeeze(eq_cp['heat_capacity'].values)

# === Extract Phase Fractions ===
phase_names = eq_cp['Phase'].values[0, 0, :, 0]  # shape: (T, vertex)
amounts = eq_cp['NP'].values[0, 0, :, 0]

# Organize phase fractions by phase
unique_phases = sorted(set(p for row in phase_names for p in row if p != ''))
phase_amounts = {phase: np.zeros_like(T) for phase in unique_phases}

for i, row in enumerate(phase_names):
    for j, phase in enumerate(row):
        if phase and not np.isnan(amounts[i, j]):
            phase_amounts[phase][i] += amounts[i, j]


# === Plotting ===

fig, axs = plt.subplots(3, 1, figsize=(6, 12), sharex=True)

# Enthalpy
axs[0].plot(T, Hm, color='cadetblue')
axs[0].set_ylabel('Enthalpy [J/mol]')
axs[0].grid(True)

# Heat Capacity
axs[1].plot(T, Cp, color='rebeccapurple')
axs[1].set_ylabel(r'$C_p$ [J/mol·K]')
axs[1].grid(True)

# Phase Fractions
for phase, data in phase_amounts.items():
    axs[2].plot(T, data, label=phase)
axs[2].set_ylabel('Phase Fraction')
axs[2].set_xlabel('Temperature (K)')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()